In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xml.etree.ElementTree as ET
sys.path.append('../../..')
from src import raw_data_path
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

In [2]:
def get_results(path_to_xml, input_concentration):
    tree = ET.parse(path_to_xml)
    _dfs = []
    for a in tree.findall("{http://www.wyatt.com/schemas/ExperimentReport}results"):
        for b in a:
            # Get mass measurements
            if b.attrib["type"] == "molar mass":
                res = {}
                res["measurement_type"] = b[0].text
                if b[1].text != "n/a":
                    res["mass"] = float(b[1].text)
                else:
                    res["mass"] = np.nan
                if "uncertainty" in b[1].attrib:
                    res["uncertainty"] = float(b[1].attrib["uncertainty"])
                else:
                    res["uncertainty"] = np.nan
                _dfs.append(pd.DataFrame(res, index=[0]))

            # Get concentration
            if b.attrib["type"] == "concentration":
                ave_conc = float(b[1].text)

            # Get total mass
            if b.attrib["type"] == "mass":
                if b[0].text == "Calculated Mass":
                    total_mass = float(b[1].text)

    df = pd.concat(_dfs, axis=0, ignore_index=True)
    df["input_concentration"] = input_concentration
    return df

### Wild type

In [3]:
res500 = get_results(
    raw_data_path + "SEC MALS/xml/ASTRA 6 Report 20220128_Par2Ring_WT_S75_500ugml.xml",
    500,
)
res750 = get_results(
    raw_data_path + "SEC MALS/xml/ASTRA 6 Report 20220128_Par2Ring_WT_S75_750ugml.xml",
    750,
)
res2000 = get_results(
    raw_data_path + "SEC MALS/xml/ASTRA 6 Report 20210927_Par2Ring_WT_S75_2mgml.xml",
    2000,
)
res5000 = get_results(
    raw_data_path + "SEC MALS/xml/ASTRA 6 Report 20210924_Par2Ring_WT_S75_5mgml.xml",
    5000,
)
res10000 = get_results(
    raw_data_path + "SEC MALS/xml/ASTRA 6 Report 20220131_Par2Ring_WT_S75_10mgml.xml",
    10000,
)
full_res = pd.concat([res500, res750, res2000, res5000, res10000])

In [4]:
# full_res

In [5]:
full_res.to_csv("mass_measurements_titration.csv")

### Plot titration

In [6]:
# import seaborn as sns

# fig, ax = plt.subplots()
# sns.scatterplot(data=full_res, x='input_concentration', y='mass', hue='measurement_type')

In [7]:
# res_sample = full_res[full_res['measurement_type'] == 'Mw']

# fig, ax = plt.subplots()
# ax.scatter(res_sample['input_concentration'], res_sample['mass'])
# ax.errorbar(res_sample['input_concentration'], res_sample['mass'], yerr=res_sample['uncertainty'])

### L109R

In [8]:
res750_L109R = get_results(
    raw_data_path + "SEC MALS/xml/ASTRA 6 Report 20210924_Par2Ring_L109R_S75_test.xml",
    750,
)
full_res_L109R = res750_L109R
full_res_L109R.to_csv("mass_measurements_L109R.csv")